In [1]:
import sys
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from bundestag.src.data.ensure_data import get_votes

In [2]:
votes = get_votes()
votes

,fraction,poll_id,name,index,vote_id,vote,fid_legislatur,fid_topic,label,date,parliament_vote,n_votes,sum_yes,sum_no,sum_abs,party_line,on_party_line,n_dissent,unanimity,y
0,AfD,4370,Kay-Uwe Ziegler,103310,425921,no,132,28,Änderung des Infektionsschutzgesetzes (Ende de...,2021-11-18,yes,736,398,254,0,no,True,3,69,106
1,AfD,4370,Joachim Wundrak,103311,425920,no,132,28,Änderung des Infektionsschutzgesetzes (Ende de...,2021-11-18,yes,736,398,254,0,no,True,7,69,106
2,AfD,4370,Christian Wirth,103313,425918,no,132,28,Änderung des Infektionsschutzgesetzes (Ende de...,2021-11-18,yes,736,398,254,0,no,True,2,69,106
3,AfD,4370,Wolfgang Wiehle,103314,425917,no,132,28,Änderung des Infektionsschutzgesetzes (Ende de...,2021-11-18,yes,736,398,254,0,no,True,0,69,106
4,AfD,4370,Harald Weyel,103315,425916,no,132,28,Änderung des Infektionsschutzgesetzes (Ende de...,2021-11-18,yes,736,398,254,0,no,True,4,69,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92130,SPD,5593,Heike Baehrens,719,574943,no,132,11,Zweite Beratung zur Aufhebung des Lieferketten...,2024-06-13,no,733,250,401,0,no,True,2,186,66
92131,SPD,5593,Johannes Arlt,720,574942,no,132,11,Zweite Beratung zur Aufhebung des Lieferketten...,2024-06-13,no,733,250,401,0,no,True,1,186,66
92132,SPD,5593,Niels Annen,721,574941,no,132,11,Zweite Beratung zur Aufhebung des Lieferketten...,2024-06-13,no,733,250,401,0,no,True,2,186,66
92133,SPD,5593,Dagmar Andres,722,574940,no,132,11,Zweite Beratung zur Aufhebung des Lieferketten...,2024-06-13,no,733,250,401,0,no,True,2,186,66


In [35]:
vote_map = {
    "yes": "#00aa00",
    "no": "#aa0000",
    "abstain": "#000000",
}

fr = "CDU"
votes_plot = votes.loc[votes.fraction.eq(fr)]

# ranges and panel sizes:
panel1_xmin = votes_plot.unanimity.max()
panel3_xmax = votes_plot.groupby("y")["on_party_line"].apply(lambda x: sum(x == False)).max()
xspan = panel1_xmin + panel3_xmax
# poll result should be 5 % width of the plot:
panel2_width = 1/50 * xspan
height = votes_plot.y.max()

In [39]:
votes_opl = votes_plot.loc[votes_plot.on_party_line]
votes_opl = votes_opl.groupby("y").agg("first").reset_index()
votes_dissent = votes_plot.loc[~votes_plot.on_party_line]
parliament_vote = votes_plot.groupby("y").parliament_vote.agg("first").to_frame().reset_index()
parliament_vote["x"] = 0

fig = make_subplots(
    cols=3,
    rows=1,
    column_widths=[panel1_xmin, panel2_width, panel3_xmax],
    horizontal_spacing=.0,
    shared_yaxes=True,
)

# single bars for the fraction majority vote:
for vote, grp in votes_opl.groupby("vote", observed=True):
    
    fig.add_trace(
        go.Bar(
            orientation="h",
            y=grp.y,
            x=-grp.unanimity,
            marker=dict(
                line_width=0,
                # line_color="white",
                color=vote_map[vote],
            ),
            showlegend=False,
            customdata=grp[["label", "date"]],
            hovertemplate="%{customdata[0]}<extra></extra>",
        ),
        col=1, row=1
    )

# individual markers for each dissenter:
for (name, vote), grp in votes_dissent.groupby(["name", "vote"], observed=True):

    fig.add_trace(
        go.Bar(
            orientation="h",
            y=grp.y,
            x=[1],
            marker=dict(
                line_width=.5,
                line_color="white",
                color=vote_map[vote],
            ),
            showlegend=False,
            customdata=grp[["label", "date", "vote", "name"]],
            hovertemplate="<b>%{customdata[3]}</b> (%{customdata[1]})<br>%{customdata[0]}<extra>%{customdata[2]}</extra>",
        ),
        col=3, row=1
    )

# overall result of each vote:
for vote, grp in parliament_vote.groupby("parliament_vote"):

    fig.add_trace(
        go.Scatter(
            mode="markers",
            y=grp.y,
            x=grp.x,
            marker_color=vote_map[vote],
            showlegend=False,
        ),
        col=2, row=1,
    )

fig.add_annotation(
    text="<= Fraktionslinie",
    x=-5, y=1,
    xanchor="right", xref="x",
    yanchor="bottom", yref="y domain",
    showarrow=False,
    font=dict(size=18),
    col=1, row=1,
)

fig.add_annotation(
    text="Dissens =>",
    x=5, y=1,
    xanchor="left", xref="x",
    yanchor="bottom", yref="y3 domain",
    showarrow=False,
    font=dict(size=18),
    col=3, row=1,
)

# white bg for parliamentary vote:
fig.add_shape(
    type="rect",
    x0=0, x1=1, xref="x2 domain",
    y0=0, y1=1, yref="y2 domain",
    col=2, row=1,
    line_width=0,
    fillcolor="white",
    layer="below",
)

fig.update_layout(
    barmode="relative",
    width=1200,
    height=1500,
    plot_bgcolor="#dddddd",
    margin=dict(t=30, r=0, b=0, l=0),
)

fig.update_xaxes(
    showticklabels=False,
    showgrid=False,
    zeroline=False,
)

fig.update_yaxes(
    showticklabels=False,
    showgrid=False,
    zeroline=False,
    range=[.5, height + .5]
)
